In [34]:
# Import dependencies
import pandas as pd

In [35]:
#Load in CSVs from http://nflsavant.com/about.php
data_2020 = "data/pbp-2020.csv"
yr2020_df = pd.read_csv(data_2020)

data_2019 = "data/pbp-2019.csv"
yr2019_df = pd.read_csv(data_2019)

data_2018 = "data/pbp-2018.csv"
yr2018_df = pd.read_csv(data_2018)

data_2017 = "data/pbp-2017.csv"
yr2017_df = pd.read_csv(data_2017)

data_2016 = "data/pbp-2016.csv"
yr2016_df = pd.read_csv(data_2016)

data_2015 = "data/pbp-2015.csv"
yr2015_df = pd.read_csv(data_2015)

data_2014 = "data/pbp-2014.csv"
yr2014_df = pd.read_csv(data_2014)

In [36]:
frames = [yr2020_df, yr2019_df, yr2018_df, yr2017_df, yr2016_df, yr2015_df, yr2014_df]
nfl_data = pd.concat(frames)
nfl_data

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards
0,2020122602,2020-12-26,2,9,44,LV,MIA,2,10,82,...,0,0,NaN,18,OPP,0,NaN,0,NaN,0
1,2020122602,2020-12-26,2,9,3,LV,MIA,3,19,73,...,0,0,LEFT GUARD,27,OPP,0,NaN,0,NaN,0
2,2020122500,2020-12-25,3,1,17,MIN,NO,3,9,67,...,0,0,NaN,33,OPP,0,NO,1,DEFENSIVE HOLDING,0
3,2020122500,2020-12-25,3,1,12,MIN,NO,1,10,82,...,0,0,NaN,18,OPP,1,NO,1,DEFENSIVE PASS INTERFERENCE,17
4,2020122500,2020-12-25,3,1,6,MIN,NO,1,1,99,...,0,0,LEFT GUARD,1,OPP,0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45690,2014122815,2014-12-28,3,11,21,SEA,LA,3,8,76,...,0,0,NaN,24,OPP,0,NaN,0,NaN,0
45691,2014122815,2014-12-28,3,11,17,SEA,LA,4,8,76,...,0,0,NaN,24,OPP,0,NaN,0,NaN,0
45692,2014122815,2014-12-28,3,11,13,SEA,LA,0,0,35,...,0,0,NaN,35,OWN,0,NaN,0,NaN,0
45693,2014122815,2014-12-28,3,9,12,SEA,LA,1,10,31,...,0,0,LEFT GUARD,31,OWN,0,NaN,0,NaN,0


In [37]:
nan_value = float("NaN")
nfl_data.replace("", nan_value, inplace=True)
nfl_data.dropna(subset = ["OffenseTeam"], inplace=True)
nfl_data

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards
0,2020122602,2020-12-26,2,9,44,LV,MIA,2,10,82,...,0,0,NaN,18,OPP,0,NaN,0,NaN,0
1,2020122602,2020-12-26,2,9,3,LV,MIA,3,19,73,...,0,0,LEFT GUARD,27,OPP,0,NaN,0,NaN,0
2,2020122500,2020-12-25,3,1,17,MIN,NO,3,9,67,...,0,0,NaN,33,OPP,0,NO,1,DEFENSIVE HOLDING,0
3,2020122500,2020-12-25,3,1,12,MIN,NO,1,10,82,...,0,0,NaN,18,OPP,1,NO,1,DEFENSIVE PASS INTERFERENCE,17
4,2020122500,2020-12-25,3,1,6,MIN,NO,1,1,99,...,0,0,LEFT GUARD,1,OPP,0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45690,2014122815,2014-12-28,3,11,21,SEA,LA,3,8,76,...,0,0,NaN,24,OPP,0,NaN,0,NaN,0
45691,2014122815,2014-12-28,3,11,17,SEA,LA,4,8,76,...,0,0,NaN,24,OPP,0,NaN,0,NaN,0
45692,2014122815,2014-12-28,3,11,13,SEA,LA,0,0,35,...,0,0,NaN,35,OWN,0,NaN,0,NaN,0
45693,2014122815,2014-12-28,3,9,12,SEA,LA,1,10,31,...,0,0,LEFT GUARD,31,OWN,0,NaN,0,NaN,0


In [38]:
nfl_data = nfl_data.drop(columns = "Challenger")
nfl_data

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards
0,2020122602,2020-12-26,2,9,44,LV,MIA,2,10,82,...,0,0,NaN,18,OPP,0,NaN,0,NaN,0
1,2020122602,2020-12-26,2,9,3,LV,MIA,3,19,73,...,0,0,LEFT GUARD,27,OPP,0,NaN,0,NaN,0
2,2020122500,2020-12-25,3,1,17,MIN,NO,3,9,67,...,0,0,NaN,33,OPP,0,NO,1,DEFENSIVE HOLDING,0
3,2020122500,2020-12-25,3,1,12,MIN,NO,1,10,82,...,0,0,NaN,18,OPP,1,NO,1,DEFENSIVE PASS INTERFERENCE,17
4,2020122500,2020-12-25,3,1,6,MIN,NO,1,1,99,...,0,0,LEFT GUARD,1,OPP,0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45690,2014122815,2014-12-28,3,11,21,SEA,LA,3,8,76,...,0,0,NaN,24,OPP,0,NaN,0,NaN,0
45691,2014122815,2014-12-28,3,11,17,SEA,LA,4,8,76,...,0,0,NaN,24,OPP,0,NaN,0,NaN,0
45692,2014122815,2014-12-28,3,11,13,SEA,LA,0,0,35,...,0,0,NaN,35,OWN,0,NaN,0,NaN,0
45693,2014122815,2014-12-28,3,9,12,SEA,LA,1,10,31,...,0,0,LEFT GUARD,31,OWN,0,NaN,0,NaN,0


In [39]:
nfl_data.dropna(how='all', axis=1, inplace=True)

In [40]:
nfl_data

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards
0,2020122602,2020-12-26,2,9,44,LV,MIA,2,10,82,...,0,0,NaN,18,OPP,0,NaN,0,NaN,0
1,2020122602,2020-12-26,2,9,3,LV,MIA,3,19,73,...,0,0,LEFT GUARD,27,OPP,0,NaN,0,NaN,0
2,2020122500,2020-12-25,3,1,17,MIN,NO,3,9,67,...,0,0,NaN,33,OPP,0,NO,1,DEFENSIVE HOLDING,0
3,2020122500,2020-12-25,3,1,12,MIN,NO,1,10,82,...,0,0,NaN,18,OPP,1,NO,1,DEFENSIVE PASS INTERFERENCE,17
4,2020122500,2020-12-25,3,1,6,MIN,NO,1,1,99,...,0,0,LEFT GUARD,1,OPP,0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45690,2014122815,2014-12-28,3,11,21,SEA,LA,3,8,76,...,0,0,NaN,24,OPP,0,NaN,0,NaN,0
45691,2014122815,2014-12-28,3,11,17,SEA,LA,4,8,76,...,0,0,NaN,24,OPP,0,NaN,0,NaN,0
45692,2014122815,2014-12-28,3,11,13,SEA,LA,0,0,35,...,0,0,NaN,35,OWN,0,NaN,0,NaN,0
45693,2014122815,2014-12-28,3,9,12,SEA,LA,1,10,31,...,0,0,LEFT GUARD,31,OWN,0,NaN,0,NaN,0


In [41]:
nfl_data.to_csv("nfldata")